In [486]:
import numpy as np
import pandas as pd
from collections import Counter

In [487]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [488]:
df = pd.read_csv('Game_of_Thrones_Script.csv')
df.head()

,Release Date,Season,Episode,Episode Title,Name,Sentence
0,2011-04-17,Season 1,Episode 1,Winter is Coming,waymar royce,What do you expect? They're savages. One lot s...
1,2011-04-17,Season 1,Episode 1,Winter is Coming,will,I've never seen wildlings do a thing like this...
2,2011-04-17,Season 1,Episode 1,Winter is Coming,waymar royce,How close did you get?
3,2011-04-17,Season 1,Episode 1,Winter is Coming,will,Close as any man would.
4,2011-04-17,Season 1,Episode 1,Winter is Coming,gared,We should head back to the wall.


In [489]:
lines = df["Sentence"].to_list()

corpus = ""
for line in lines:
    corpus += (line.lower() + "\n")

print(corpus[:500])



what do you expect? they're savages. one lot steals a goat from another lot and before you know it, they're ripping each other to pieces.
i've never seen wildlings do a thing like this. i've never seen a thing like this, not ever in my life.
how close did you get?
close as any man would.
we should head back to the wall.
do the dead frighten you?
our orders were to track the wildlings. we tracked them. they won't trouble us no more.
you don't think he'll ask us how they died? get back on your hor


In [490]:
import re
corpus = re.split("([^a-zA-Z0-9\w'])",corpus)
corpus = [x for x in corpus if x != ' ' and x != '']
word_counts = Counter(corpus)
sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
int_to_vocab = {k:w for k, w in enumerate(sorted_vocab)}
vocab_to_int = {w:k for k, w in int_to_vocab.items()}
n_vocab = len(int_to_vocab)


print(n_vocab)
print(len(corpus))



9889
368745


In [491]:
seq_size=16
batch_size=64
int_text = [vocab_to_int[w] for w in corpus]
num_batches = int(len(int_text) / (seq_size * batch_size))
in_text = int_text[:num_batches * batch_size * seq_size]
out_text = np.zeros_like(in_text)
out_text[:-1] = in_text[1:]
out_text[-1] = in_text[0]
in_text = np.reshape(in_text, (batch_size, -1))
out_text = np.reshape(out_text, (batch_size, -1))

In [492]:
def get_batches(in_text, out_text, batch_size, seq_size):
    num_batches = np.prod(in_text.shape) // (seq_size * batch_size)
    for i in range(0, num_batches * seq_size, seq_size):
        yield in_text[:, i:i+seq_size], out_text[:, i:i+seq_size]

Define network

In [493]:
class RNNModule(nn.Module):
    def __init__(self, n_vocab, seq_size, embedding_size, hidden_dim):
        super(RNNModule, self).__init__()
        self.seq_size = seq_size
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(n_vocab, embedding_size)
        self.lstm = nn.LSTM(embedding_size,
                            hidden_dim,
                            num_layers = 2,
                            batch_first=True)
        self.dense = nn.Linear(hidden_dim, n_vocab)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.dense(output)

        return logits, state
    def zero_state(self, batch_size):
        return (torch.zeros(1, batch_size, self.hidden_dim),
                torch.zeros(1, batch_size, self.hidden_dim))

Define loss function

In [494]:
def get_loss_and_train_op(net, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    return criterion, optimizer

Predict function

In [495]:
def predict(device, net, words, n_vocab, vocab_to_int, int_to_vocab, top_k=5):
    net.eval()

    state_h, state_c = net.zero_state(1)
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    for w in words:
        ix = torch.tensor([[vocab_to_int[w]]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))
    
    _, top_ix = torch.topk(output[0], k=top_k)
    choices = top_ix.tolist()
    choice = np.random.choice(choices[0])

    words.append(int_to_vocab[choice])
    for _ in range(150):
        ix = torch.tensor([[choice]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))

        _, top_ix = torch.topk(output[0], k=top_k)
        choices = top_ix.tolist()
        choice = np.random.choice(choices[0])
        words.append(int_to_vocab[choice])

    print(' '.join(words))

Training

In [496]:
gradients_norm = 5
embedding_size=64
hidden_dim=256
def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    net = RNNModule(n_vocab, seq_size,
                    embedding_size, hidden_dim)
    net = net.to(device)

    criterion, optimizer = get_loss_and_train_op(net, 0.01)

    iteration = 0
    for e in range(200):
        batches = get_batches(in_text, out_text, batch_size, seq_size)
        state_h, state_c = net.zero_state(batch_size)
        
        # Transfer data to GPU
        state_h = state_h.to(device)
        state_c = state_c.to(device)
        for x, y in batches:
            iteration += 1
            
            # Tell it we are in training mode
            net.train()

            # Reset all gradients
            optimizer.zero_grad()

            # Transfer data to GPU
            x = torch.tensor(x).to(device)
            y = torch.tensor(y).to(device)

            logits, (state_h, state_c) = net(x, (state_h, state_c))
            loss = criterion(logits.transpose(1, 2), y.long())

            state_h = state_h.detach()
            state_c = state_c.detach()

            loss_value = loss.item()

            # Perform back-propagation
            loss.backward()
        
            _ = torch.nn.utils.clip_grad_norm_(
                net.parameters(), gradients_norm)

            # Update the network's parameters
            optimizer.step()
            if iteration % 100 == 0:
                print('Epoch: {}/{}'.format(e, 200),
                      'Iteration: {}'.format(iteration),
                      'Loss: {}'.format(loss_value))

            if iteration % 1000 == 0:
                test_initial_words = ["i", "am"]
                predict(device, net, test_initial_words, n_vocab,
                        vocab_to_int, int_to_vocab, top_k=5)
                torch.save(net.state_dict(),
                           'models/model-{}.pth'.format(iteration))   

In [497]:
main()

AttributeError: 'RNNModule' object has no attribute 'lstm_size'

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
saved_model = RNNModule(n_vocab, seq_size, embedding_size, hidden_dim)
saved_model.load_state_dict(torch.load('./models/model-18000.pth'))
saved_model = saved_model.cuda()
initial_words = ["i", "am"]
predict(device, saved_model, initial_words, n_vocab, vocab_to_int, int_to_vocab, top_k=5)

i am not to saying his thoughts with a small day , but he had one fat go to the stormlands and watch has the unsullied while they had lost . they chose the king of unsullied . we will wait until they beat us alive the day on top , we might . i regret them for other girls . and we have an arrow . and when you were serving your father - - i was no crime i cried when he had his pyromancer - mad . tyrion was the key . the king is so much what you did ? i'm going to give me the eyrie ? why is she still in king's hand and i betrayed you doing with me , do i ? what happens my father after the king that matters that took up the knights of your fathers . 
 the eyrie . i
